## Логистическая регрессия. Log Loss

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

### Задание 1.
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

__Наша функция, которую необходимо откорректировать:__

In [3]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [4]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

__Новая функция calc_logloss:__

In [5]:
def calc_logloss(y, y_pred):
    y_pred = y_pred.copy()
    for i in range(len(y_pred)):
        if y_pred[i] == 0.0:
            y_pred[i] = 0.000000001
        elif y_pred[i] == 1.0:
            y_pred[i] = 0.999999999
    score = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return score

__Проверяем:__

In [6]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025832891313

### Задание 2.
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [7]:
def calc_std_feat(x):
    res = (X - X.mean()) / X.std()
    return res

In [10]:
X_st = X[1:,].copy()
X_mean, X_std = X_st.mean(axis=1).reshape(-1,1), X_st.std(axis=1).reshape(-1,1)
X[1:,] = (X_st - X_mean) / X_std

In [11]:
X_st

array([[-0.57142857, -0.57142857, -0.21428571, -0.57142857,  0.14285714,
        -0.92857143,  0.85714286,  2.64285714, -0.57142857, -0.21428571],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064]])

In [12]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [13]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    return W

In [14]:
def param_search(X, y, iterations, alpha):
    best_errors = np.inf
    best_params = []
    weights = []
    err = 0
    y_pred = []
    for i in iterations:
        for alpha_ in alpha:
            weights = eval_model(X, y, i, alpha_)
            y_pred = sigmoid(np.dot(weights, X))
            err = calc_logloss(y, y_pred)
            if err < best_errors:
                best_errors = err
                best_params = [i, alpha_]
    return best_errors, best_params

In [15]:
iterations = [1000, 2000, 10000, 15000, 40000, 100000, 500000, 1000000]
alpha = [0.1, 0.05, 0.01, 0.001, 0.0001, 0.00001]

In [16]:
best_errors, best_params = param_search(X_st, y, iterations, alpha)
best_errors, best_params

(0.2997021777181629, [100000, 0.1])

### Задание 3.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [17]:
W = eval_model(X_st, y, 100000, 0.1)
W

array([-4.75776597, -2.22586568,  7.26557831])

In [18]:
def calc_pred_proba(W, X):
    y_pred = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred

In [19]:
calc_pred_proba(W, X_st)

array([0.16621751, 0.07373019, 0.99217942, 0.11187659, 0.48267444,
       0.21575294, 0.9963354 , 0.00556411, 0.20050263, 0.97576792])

### Задание 4.
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [20]:
def calc_pred(W, X):
    
    y_pred_class_ = np.sign(np.dot(W, X))
    y_pred_class = np.empty_like(y_pred_class_)
    
    for i in range(len(y_pred_class_)):
        
        if y_pred_class_[i] == -1.0:
            y_pred_class[i] = 0
        elif y_pred_class_[i] == 1.0:
            y_pred_class[i] = 1
    
    return y_pred_class

In [21]:
y_pred = calc_pred(W, X_st)
y_pred

array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1.])

### Задание 5.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [22]:
def accuracy(y, y_pred):
    return (y_pred==y).sum()/len(y)

def err_matrix(y, y_pred):
    errors = np.zeros((2,2))
    for real, pred in zip(y, y_pred):
        if real == pred:
            if real == 1:
                errors[0,0] += 1
            else:
                errors[1,1] += 1
        else:
            if real == 1:
                errors[1,0] += 1
            else:
                errors[0,1] += 1
    return errors

In [23]:
accuracy = accuracy(y, y_pred)
accuracy

0.8

In [24]:
matrix = err_matrix(y, y_pred)
matrix

array([[3., 0.],
       [2., 5.]])

In [25]:
precision = matrix[0,0] / (matrix[0,0] + matrix[0,1])
precision

1.0

In [26]:
recall = matrix[0,0] / (matrix[0,0] + matrix[1,0])
recall

0.6

In [27]:
F1 = 2 * precision * recall / (precision + recall)
F1

0.7499999999999999

### Задание 6.
Могла ли модель переобучиться? Почему?

Да, модель может переобучиться. Переобучение возможно так в исходной матрице малое количество объектов, из за чего обучение происходит быстрее, а также не были использовани $L_1$ или $L_2$ регуляции

### Задание 7.
(опция) Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [40]:
def eval_model_l1(X,y, iterations, alpha=1e-4, lambda_=1e-3):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    for i in range(iterations):
        y_pred = sigmoid(W.dot(X))
        W -= alpha*(1/X.shape[1]*np.dot(y_pred-y, X.T)+lambda_*np.sign(W))
    return W

In [41]:
eval_model_l1(X_st, y, 100000, 0.01)

array([-4.15353758, -1.97483301,  6.55686416])

In [42]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-3):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    for i in range(iterations):
        y_pred = sigmoid(W.dot(X))
        W -= alpha*(1/X.shape[1]*np.dot(y_pred-y, X.T)+lambda_*W)
    return W

In [43]:
eval_model_l2(X_st, y, 100000, 0.01)

array([-3.44143488, -1.6577445 ,  5.63483495])